# Camphr: spaCy plugin for Transformers, Udify, Elmo, KNP

[PKSHA Technology](https://pkshatech.com/ja/) から，
NLPライブラリ[Camphrを公開しました](https://github.com/PKSHATechnology-Research/camphr). 
[Transformers](https://github.com/huggingface/transformers)や[Udify](https://github.com/Hyperparticle/udify)などの最先端の手法の他，[knp](http://nlp.ist.i.kyoto-u.ac.jp/?KNP)など伝統的な言語処理手法を[spaCy](https://spacy.io/)上で組み合わせて使うことができます．
本記事では，Camphrの特徴や，簡単な使い方を紹介していきます．

# spaCy

Camphrは[spaCy](https://spacy.io/)のプラグインです．
日本だと[Ginza](https://github.com/megagonlabs/ginza)がspaCyを利用しており有名ですね．

spaCyはNLPフレームワークで，以下のような特長があります.

1. 様々な機能を合成できる (深層学習からパターンマッチまで何でもOK)
2. モデルを1コマンドで保存&復元できる

特に1つ目の機能は実運用上とても重要です．
NLPはここ数年で大幅に進歩しましたが，実社会のタスクはend-to-endにデータを食わせればOK，みたいに美味しいものばかりではありません.
かといって新しい手法を全く使わないのも，あまり未来がなさそうです．
spaCyを使うと，最新の手法からルールベースの手法まで，様々な手法を組み合わせることができます．
そしてCamphrを使うと，例えばBERTをfine-tuneした後にKNPと正規表現を組み合わせる，ということが簡単にできます．

そして2つめの機能のお陰で，組み合わせた複雑なパイプラインを簡単に復元し，デプロイすることができます．

# Quick Tour

日本語を使う場合は，MeCabをインストールしておく必要があります．
インストールはpipでできます. 

```
$ pip install camphr[mecab]
```

## Transformersの埋め込みベクトルを使う

まずはBERTの埋め込みベクトルを使ってみましょう．[Bert as a service](https://github.com/hanxiao/bert-as-service)と同じです．  
下のように，パイプラインの設定を`camphr.load`に渡しましょう．
例はYamlですが，[omegaconf](https://github.com/omry/omegaconf)が処理できるものならば何でも良いです. (dict, json等)

In [26]:
import camphr

nlp = camphr.load(
    """
lang:
    name: ja_mecab
pipeline:
    transformers_model:
        trf_name_or_path: bert-base-japanese
"""
)

裏で[transformers](https://github.com/huggingface/transformers)が動いており，パラメータ等は勝手にダウンロードされます．

In [27]:
doc = nlp("BERTの埋め込みベクトルをspacyで使う")
print(doc.tensor)
token = doc[0]
print(token, token.vector) # トークンのベクトル
span = doc[1:4]
print(span, span.vector) # スパンのベクトル

tensor([[ 1.3047, -1.0605,  0.4627,  ..., -0.6162, -0.3270,  0.1817],
        [ 0.2818, -0.6007, -0.3528,  ...,  0.1504,  0.3475,  0.0229],
        [ 1.0175, -0.7950, -0.9003,  ..., -0.5052,  0.1553, -0.3837],
        ...,
        [-0.1360, -0.1076, -0.1391,  ...,  0.1577, -0.3162,  0.1373],
        [ 0.2194, -0.8604,  0.5441,  ..., -0.3768, -0.2112, -0.1784],
        [ 0.5203, -0.6748,  0.4415,  ..., -0.1184, -0.4140,  0.4811]])
BERT [ 1.30469453e+00 -1.06053877e+00  4.62672085e-01 -1.38373268e+00
 -1.95268679e+00 -3.93442586e-02 -9.07721579e-01  8.44051391e-02
  1.04509687e+00 -1.91310287e+00  6.52137756e-01 -8.92923057e-01
 -1.12373018e+00 -1.65434408e+00  1.25064409e+00  1.23973083e+00
  4.46228176e-01 -1.09267545e+00  1.24803036e-02  7.96998441e-01
 -9.89600778e-01 -9.65374708e-01  3.18519175e-01  1.98907554e-01
  9.09519315e-01  3.10369104e-01  6.17856383e-01 -1.10153124e-01
  2.71469235e-01 -1.49982110e-01  2.68634844e+00 -4.32227671e-01
  6.76543891e-01  9.81138587e-01  8.79047

コサイン類似度は以下のようにして取得します．

In [28]:
doc2 = nlp("BERTは自然言語処理を大幅に進歩させました")
print("文の類似度     : ", doc.similarity(doc2))
print("トークンの類似度: ", doc[0].similarity(doc2[0]))

文の類似度     :  0.8219287395477295
トークンの類似度:  0.8147419691085815


他のモデルや，自作モデルを使いたい，場合は[ドキュメント](https://camphr.readthedocs.io/en/latest/notes/transformers.html)を参照してください

# Transformersのfine-tuning

以下のように，CLIを使って簡単にfine-tuneできます．

```bash
$ camphr train train.data.path="./train.jsonl" \ # トレーニングデータ
               textcat_label="./label.json" \    # ラベル
               pretrained=bert-base-cased  \     # 事前学習モデル名
               lang=ja_mecab                     # spacyの言語名
```